In [1]:
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras import regularizers
import keras

Using TensorFlow backend.


In [0]:
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY
 

In [0]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [0]:
# define cnn model
def define_model(img_size=32):
	batch_size=128
	weight_decay=1e-4
	model = Sequential()
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(weight_decay), kernel_initializer='he_uniform', padding='same', input_shape=(img_size, img_size, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(BatchNormalization())
	model.add(Dropout(0.2))
 
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(weight_decay), kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(BatchNormalization())
	model.add(Dropout(0.3))

	model.add(Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(weight_decay), kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(BatchNormalization())
	model.add(Dropout(0.4))

	model.add(Flatten())
	model.add(Dense(4096, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dropout(0.5))
	model.add(Dense(10, activation='softmax'))
	# compile model
	model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
	#opt = SGD(lr=0.001, momentum=0.3)
	#model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model


In [0]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['acc'], color='blue', label='train')
	pyplot.plot(history.history['val_acc'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

In [9]:

# load dataset
trainX, trainY, testX, testY = load_dataset()
# prepare pixel data
trainX, testX = prep_pixels(trainX, testX)
# define model
model = define_model()
# create data generator
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

# Apply the generator
datagen.fit(trainX)

# prepare iterator
it_train = datagen.flow(trainX, trainY, batch_size=128)
# fit model
steps = int(trainX.shape[0] / 64)
history = model.fit_generator(it_train, steps_per_epoch=len(trainX) / 32, epochs=15, validation_data=(testX, testY), verbose=1)
# evaluate model
_, acc = model.evaluate(testX, testY, verbose=0)
print('> %.3f' % (acc * 100.0))
# learning curves
summarize_diagnostics(history)



Epoch 1/15
1563/1562 [==============================] - 148s 95ms/step - loss: 1.5046 - acc: 0.5405 - val_loss: 1.0162 - val_acc: 0.6886
Epoch 2/15
1563/1562 [==============================] - 144s 92ms/step - loss: 1.0313 - acc: 0.6829 - val_loss: 0.9594 - val_acc: 0.7189
Epoch 3/15
1563/1562 [==============================] - 143s 92ms/step - loss: 0.9048 - acc: 0.7318 - val_loss: 0.8274 - val_acc: 0.7657
Epoch 4/15
1563/1562 [==============================] - 142s 91ms/step - loss: 0.8221 - acc: 0.7624 - val_loss: 0.7322 - val_acc: 0.8017
Epoch 5/15
1563/1562 [==============================] - 143s 92ms/step - loss: 0.7623 - acc: 0.7837 - val_loss: 0.7749 - val_acc: 0.7888
Epoch 6/15
1563/1562 [==============================] - 143s 91ms/step - loss: 0.7112 - acc: 0.8000 - val_loss: 0.6607 - val_acc: 0.8248
Epoch 7/15
1563/1562 [==============================] - 143s 92ms/step - loss: 0.6720 - acc: 0.8134 - val_loss: 0.8130 - val_acc: 0.7678
Epoch 8/15
1563/1562 [===================